In [25]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [32]:
def scrape_top_news_from_page(url):
    try:
        url = 'https://news.naver.com/main/ranking/popularDay.naver'
        headers = {'User-Agent':'Mozilla/5.0'}
        response = requests.get(url, headers = headers)
        response.raise_for_status() 

        soup = BeautifulSoup(response.text, 'html.parser')

        top_news_list = []

        for media_block in soup.select('.rankingnews_box')[:3]:
            media_name_element = media_block.select_one('.rankingnews_name')
            if media_name_element:
                media_name = media_name_element.get_text(strip=True)
                news_list = []

                for news_item in media_block.select('li a')[:5]:
                    news_title = news_item.get_text(strip=True)
                    news_link = news_item['href']

                    news_content = get_article_content(news_link, headers)
                    news_list.append({'title': news_title, 'link': news_link, 'content': news_content})

                top_news_list.append({'press': media_name, 'news': news_list})
        return top_news_list
    
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

In [33]:
def get_article_content(url, headers):
    try:
        response = requests.get(url, headers= {'User-Agent':'Mozilla/5.0'})
        response.raise_for_status()  # 에러 발생 시 예외 처리

        # HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')

        # 기사 내용 가져오기
        article_body_element = soup.select_one('._article_content')
        if article_body_element:
            content = article_body_element.get_text('\n', strip=True)
            return content
        else:
            print(f"Error: Could not find article content on {article_url}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

In [34]:
given_page_url = 'https://news.naver.com/main/ranking/popularDay.naver'

top_news_result = scrape_top_news_from_page(given_page_url)

if top_news_result:
    for item in top_news_result:
        print(f"\n언론사: {item['press']}")
        for idx, news in enumerate(item['news'], start=1):
            print(f"\n기사 {idx}: {news['title']}")
            print(f"링크: {news['link']}")
            print(f"내용:\n{news['content'][:300]}...")  # 기사 내용 중 일부만 출력
else:
    print("뉴스를 가져올 수 없습니다.")


언론사: 한국경제

기사 1: 빅뱅 GD '마약 혐의' 입건…"이선균 사건과 별개"
링크: https://n.news.naver.com/article/015/0004906744?ntype=RANKING
내용:
사진=뉴스1
배우 이선균이 마약 투약 혐의로 입건된 가운데, 이번엔 그룹 빅뱅 출신의 GD(권지용)가 추가 입건됐다.
25일 법조계와 경찰에 따르면 인천경찰청 마약범죄수사계는 마약류 관리에 관한 법률 위반 혐의로 권 씨를 불구속 입건했다.
이 씨 입건 후 강제수사를 벌여온 경찰은 권 씨를 특정해 조사한 후 그를 입건해 수사를 이어가는 중이다.
권 씨 입건과 관련해 경찰은 "수사 중인 사안이라 구체적인 내용은 밝힐 수 없다"는 입장이다.
다만 경찰은 권 씨의 경우 이 씨의 사건과는 별건이라고 설명했다.
과거 그룹 빅뱅의 리더였던 권 ...

기사 2: 
링크: https://n.news.naver.com/article/015/0004906744?ntype=RANKING
내용:
사진=뉴스1
배우 이선균이 마약 투약 혐의로 입건된 가운데, 이번엔 그룹 빅뱅 출신의 GD(권지용)가 추가 입건됐다.
25일 법조계와 경찰에 따르면 인천경찰청 마약범죄수사계는 마약류 관리에 관한 법률 위반 혐의로 권 씨를 불구속 입건했다.
이 씨 입건 후 강제수사를 벌여온 경찰은 권 씨를 특정해 조사한 후 그를 입건해 수사를 이어가는 중이다.
권 씨 입건과 관련해 경찰은 "수사 중인 사안이라 구체적인 내용은 밝힐 수 없다"는 입장이다.
다만 경찰은 권 씨의 경우 이 씨의 사건과는 별건이라고 설명했다.
과거 그룹 빅뱅의 리더였던 권 ...

기사 3: "유아인은 100억 배상"…이선균, 위약금 후폭풍 몰아치나
링크: https://n.news.naver.com/article/015/0004906433?ntype=RANKING
내용:
김헌식 대중문화평론가
"이선균, 이미지 워낙 좋아 광고 많아"
"후폭풍 굉장히 크고 강하게 불 것"
배우 유아인-이선균/ 사진=텐아시아DB
배우 이선균이 